In [ ]:
# Ingredients to be used in 10 recipes

# ["bacon", "eggs", "buns", "tomatoes", "mayonnaise"]
# ["bay scallops", "butter", "onions", "parsley", "lemons", "bread crumbs"]
# ["rice", "chicken", "cream of mushroom soup", "cream of chicken soup", "onion soup", "water"]
# ["coriander", "allspice", "cumin", "ground ginger", "turmeric", "cinnamon", "cayenne", "pepper", "salt", "lemon juice", "olive oil", "garlic", "chicken", "pita", "red onion", "parsley", "shredded red cabbage", "tahini"]
# ["sour cream", "heavy cream", "baking powder", "kosher salt", "sugar", "ground black pepper", "flour", "butter", "salt", "cutter"]
# ["hamburger", "egg", "bread crumbs", "onion", "milk", "salt", "pepper", "bacon", "catsup"]
# ["stew meat", "cabbage", "onion", "green bell peppers", "oil", "tomatoes", "garlic", "soy sauce"]
# ["chicken broth", "water", "chicken", "onion", "vegetables", "garlic", "salt", "fresh cilantro", "potatoes", "vegetable oil"]
# ["pork sparerib", "black bean sauce", "Chinese rice wine", "cornstarch", "ginger", "garlic", "ground black pepper", "cooking oil", "sesame oil", "sugar", "red chili pepper"]
# ["shell pasta", "red pepper", "green onion", "seafood", "peas", "olive oil", "sesame oil", "rice vinegar", "lemon", "mayonnaise"]

In [13]:
import pandas as pd
import os

from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

from sqlalchemy import create_engine
import mysql.connector

In [2]:

current_path = os.getcwd()
print(current_path)

os.chdir('../dataset/')
path_base_dataset = os.getcwd()
print(path_base_dataset)

os.chdir('../exploratory_data_analysis/')
path_base_notebook = os.getcwd()
print(path_base_notebook)

NUM_DATA_SAMPLES = 50000
file_name = '/sampled_dataset_' + str(NUM_DATA_SAMPLES) + '.csv'
sampled_data_file_path = path_base_dataset + file_name


c:\Batool_Data (2024)\Privat\BT_Related to Durham College\Semester #3\github\Personalized_Recipe_Generator\exploratory_data_analysis
c:\Batool_Data (2024)\Privat\BT_Related to Durham College\Semester #3\github\Personalized_Recipe_Generator\dataset
c:\Batool_Data (2024)\Privat\BT_Related to Durham College\Semester #3\github\Personalized_Recipe_Generator\exploratory_data_analysis


In [3]:
df = pd.read_csv(sampled_data_file_path)
df.head()

,Unnamed: 0,ID,name,ingredients,procedure,link,source,NER
0,1254789,1254789,"Bacon, Lettuce, Tomato And Fried Egg Sandwiches","[""4 slices bacon, halved"", ""4 None eggs"", ""4 N...","[""Lightly coat a nonstick frying pan and place...",recipes-plus.com/api/v2.0/recipes/37658,Gathered,"[""bacon"", ""eggs"", ""buns"", ""tomatoes"", ""mayonna..."
1,2162678,2162678,Sauteed Bay Scallops Recipe,"[""2 pound bay scallops"", ""1/4 pound butter, me...","[""Saute/fry onions and parsley in butter."", ""R...",cookeatshare.com/recipes/sauteed-bay-scallops-...,Recipes1M,"[""bay scallops"", ""butter"", ""onions"", ""parsley""..."
2,832049,832049,No Fuss No Mess Chicken And Rice,"[""1 c. uncooked rice"", ""1 chicken, cut up"", ""1...","[""Place rice in oblong pan."", ""Place chicken o...",www.cookbooks.com/Recipe-Details.aspx?id=334186,Gathered,"[""rice"", ""chicken"", ""cream of mushroom soup"", ..."
3,1396186,1396186,Baked Chicken Shawarma,"[""2 teaspoons coriander"", ""1 teaspoon allspice...","[""In a large bowl combine coriander, allspice,...",food52.com/recipes/41562-baked-chicken-shawarma,Gathered,"[""coriander"", ""allspice"", ""cumin"", ""ground gin..."
4,1322877,1322877,Salt-And-Pepper Biscuits,"[""3/4 cup sour cream"", ""4 tablespoons heavy cr...","[""Preheat oven to 425\u00b0F. Whisk sour cream...",www.epicurious.com/recipes/food/views/salt-and...,Gathered,"[""sour cream"", ""heavy cream"", ""baking powder"",..."


In [4]:
# # Database connection details
server = 'rag_application_server'
prg_database_name = 'personalized_recipe_generator'
user = 'root'
password = 'root'
host="localhost"
port="3306"

In [5]:
# Connect to MySQL (without specifying a database)
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password
)

cursor = connection.cursor()

# Create a new database
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {prg_database_name};")
print(f"Database {prg_database_name} created successfully.")

# Commit changes and close connection
connection.commit()
cursor.close()
connection.close()

# Create a connection engine using SQLAlchemy
db_uri = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{prg_database_name}"
engine = create_engine(db_uri)

# Write the DataFrame to the MySQL database as a table
table_name = 'recipe_knowledge_base'  # Name the table you want to create in the database
df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

print(f"Table {table_name} populated successfully.")

# # Now connect to the newly created database using LangChain's SQLDatabase library
# db_url = f"mysql+mysqlconnector://{user}:{password}@{host}/{new_database_name}"
# db = SQLDatabase.from_uri(db_url)

# # Example: Check tables in the new database (should be empty initially)
# print(db.get_table_names())  # Outputs an empty list as the new database is empty


Database personalized_recipe_generator created successfully.
Table recipe_knowledge_base populated successfully.


In [6]:
def init_database(user: str, password: str, host: str, port: str, database: str) -> SQLDatabase:
  db_uri = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
  return SQLDatabase.from_uri(db_uri)

# # Now, you can interact with this table using LangChain's SQLDatabase
db = init_database(
  user="root",
  password="root",
  host="localhost",
  port="3306",
  database=prg_database_name
  )

In [9]:
# Verify by printing the list of tables in the database
print(db.dialect)
print(db.get_table_names())  # This should list the newly created table


mysql
['recipe_knowledge_base']


In [10]:
db.run("SELECT * FROM recipe_knowledge_base LIMIT 10;")

'[(1254789, 1254789, \'Bacon, Lettuce, Tomato And Fried Egg Sandwiches\', \'["4 slices bacon, halved", "4 None eggs", "4 None hamburger buns, toasted", "4 None lettuce leaves", "2 None tomatoes, sliced", "None None mayonnaise or barbecue sauce"]\', \'["Lightly coat a nonstick frying pan and place over high heat. Fry bacon for 3 mins, turning, or until browned. Transfer to a plate and keep warm.", "Reduce heat to medium-low. Fry eggs until cooked to your liking.", "Top bottom burger buns with lettuce, bacon, tomato and egg. Add sauce of your...\', \'recipes-plus.com/api/v2.0/recipes/37658\', \'Gathered\', \'["bacon", "eggs", "buns", "tomatoes", "mayonnaise"]\'), (2162678, 2162678, \'Sauteed Bay Scallops Recipe\', \'["2 pound bay scallops", "1/4 pound butter, melted", "1 c. finely minced onions", "1/2 bunch parsley, finely minced", "Juice of 2 lemons", "1 c. bread crumbs"]\', \'["Saute/fry onions and parsley in butter.", "Remove from heat.", "Pat scallops in pan till all sides are full c

In [11]:
db.get_table_info()

'\nCREATE TABLE recipe_knowledge_base (\n\t`Unnamed: 0` BIGINT, \n\t`ID` BIGINT, \n\tname TEXT, \n\tingredients TEXT, \n\t`procedure` TEXT, \n\tlink TEXT, \n\tsource TEXT, \n\t`NER` TEXT\n)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci\n\n/*\n3 rows from recipe_knowledge_base table:\nUnnamed: 0\tID\tname\tingredients\tprocedure\tlink\tsource\tNER\n1254789\t1254789\tBacon, Lettuce, Tomato And Fried Egg Sandwiches\t["4 slices bacon, halved", "4 None eggs", "4 None hamburger buns, toasted", "4 None lettuce leaves",\t["Lightly coat a nonstick frying pan and place over high heat. Fry bacon for 3 mins, turning, or unt\trecipes-plus.com/api/v2.0/recipes/37658\tGathered\t["bacon", "eggs", "buns", "tomatoes", "mayonnaise"]\n2162678\t2162678\tSauteed Bay Scallops Recipe\t["2 pound bay scallops", "1/4 pound butter, melted", "1 c. finely minced onions", "1/2 bunch parsley\t["Saute/fry onions and parsley in butter.", "Remove from heat.", "Pat scallops in pan till all sides\tcookea

In [14]:
template = """
    You are a world's renowned chef, who can create a dish with any ingredients that are given to you.
    You have a tremendous experience in a varitey of cuisines, like for example continental, indian, mexican,
    chinese, vetnamise, thai, middle eastern, turkish, lebanese, italian, spanish, french, japanese, british, 
    and the list continues.
    You have expertize in making both vegetarian and non-vegetarian dishes. Expert meat as well as fish and seafood cook.
    You are well versed about different how to cater for gluten free or lactose free diets.

    Use all the knowledge you have to create a new, delicious recipe from the ingredients chosen by the user.

    Based on the table schema below, write a SQL query that would answer the user's question. 
    Take the conversation history into account.

    <SCHEMA>{schema}</SCHEMA>

    Conversation History: {chat_history}

    Write only the SQL query and nothing else. Do not wrap the SQL query in any other text, 
    not even backticks.

    For example:
    Question: Need a recipe using chicken, tomato, basil
    SQL Query: SELECT * 
               FROM recipe_knowledge_base 
               WHERE NER LIKE CONCAT('%', 'chicken', '%') 
               AND NER LIKE CONCAT('%', 'tomato', '%') 
               AND NER LIKE CONCAT('%', 'basil', '%');
    Question: Do you have recipes for chicken?
    SQL Query: SELECT * 
               FROM recipe_knowledge_base 
               WHERE NER LIKE '%chicken%';
    Question: Name 10 dishes
    SQL Query: SELECT name FROM recipe_knowledge_base LIMIT 10;

    Your turn:

    Question: {question}
    SQL Query:
"""

prompt = ChatPromptTemplate.from_template(template)


In [15]:
prompt.format(schema="my schema", question="Need a recipe using chicken, tomato, basil")

"Human: \n    You are a world's renowned chef, who can create a dish with any ingredients that are given to you.\n    You have a tremendous experience in a varitey of cuisines, like for example continental, indian, mexican,\n    chinese, vetnamise, thai, middle eastern, turkish, lebanese, italian, spanish, french, japanese, british, \n    and the list continues.\n    You have expertize in making both vegetarian and non-vegetarian dishes. Expert meat as well as fish and seafood cook.\n    You are well versed about different how to cater for gluten free or lactose free diets.\n\n    Use all the knowledge you have to create a new, delicious recipe from the ingredients chosen by the user.\n\n    Based on the table schema below, write a SQL query that would answer the user's question. \n    Take the conversation history into account.\n\n    <SCHEMA>my schema</SCHEMA>\n\n    Write only the SQL query and nothing else. Do not wrap the SQL query in any other text, \n    not even backticks.\n\n 

In [16]:
def get_schema(_):
    schema = db.get_table_info()
    return schema

In [17]:


llm = ChatOpenAI(temperature=0, 
                 model="gpt-4o-mini")

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\SQL Result:"])
    | StrOutputParser()
)

In [18]:
user_question = "Need a recipe using chicken, tomato, basil"
sql_chain.invoke({"question": user_question})

"SELECT * \nFROM recipe_knowledge_base \nWHERE NER LIKE CONCAT('%', 'chicken', '%') \nAND NER LIKE CONCAT('%', 'tomato', '%') \nAND NER LIKE CONCAT('%', 'basil', '%');"

In [19]:
full_chain_template = """
        You are a world's renowned chef, who can create a dish with any ingredients that are given to you.
        You have a tremendous experience in a varitey of cuisines, like for example continental, indian, mexican,
        chinese, vetnamise, thai, middle eastern, turkish, lebanese, italian, spanish, french, japanese, british, 
        and the list continues.
        You have expertize in making both vegetarian and non-vegetarian dishes. Expert meat as well as fish and seafood cook.
        You are well versed about different how to cater for gluten free or lactose free diets.

        Use all the knowledge you have to create a new, delicious recipe from the ingredients chosen by the user.
        There are going to be 8 or less ingredients given to you by the user. You also have access to some pantry staples like
        cream, eggs, butter, milk, vinegar, sugar and flour. 
        You are allowed to mix and match steps, thoughts, and ideas from your knowledge. Do not copy and paste 
        the recipe directly from the database. Show innovation and creativity in the final dish.
        
        Based on the table schema below, question, sql query, and sql response, write a natural language response.
        The natural language response should not have any mention of SQL response and SQL query.
        
        <SCHEMA>{schema}</SCHEMA>
    
        SQL Query: <SQL>{query}</SQL>
        User Question: {question}
        SQL Response: {response}
        """

prompt_response = ChatPromptTemplate.from_template(full_chain_template)

In [20]:
def run_query(query):
    return db.run(query)

In [21]:
run_query("SELECT * \nFROM recipe_knowledge_base \nWHERE NER LIKE CONCAT('%', 'chicken', '%') \nAND NER LIKE CONCAT('%', 'tomato', '%') \nAND NER LIKE CONCAT('%', 'basil', '%');")

'[(1112082, 1112082, \'Chicken Parmesan Burger\', \'["1 lb ground chicken", "1 tablespoon olive oil, plus more to fry", "2 tablespoons tomato sauce", "2 teaspoons dried basil", "1 tablespoon onion, Minced", "kosher salt", "fresh ground black pepper", "red pepper flakes", "1 cup panko breadcrumbs", "1/2 cup parmesan cheese, Grated", "mozzarella...\', \'["Mix chicken, olive oil, tomato sauce, basil, onion, kosher salt, black pepper and red pepper flakes together.", "Form the chicken mixture into balls; the texture is gloppy.", "Flatten into burger patties.", "Dip patties into a mixture of Panko breadcrumbs and Parmesan cheese.", "Place burgers...\', \'www.food.com/recipe/chicken-parmesan-burger-479163\', \'Gathered\', \'["ground chicken", "olive oil", "tomato sauce", "basil", "onion", "kosher salt", "fresh ground black pepper", "red pepper", "breadcrumbs", "parmesan cheese", "mozzarella cheese", "rolls"]\'), (1203232, 1203232, \'Chicken Cacciatore\', \'["flour, to coat chicken", "2 whole

In [22]:
full_sql_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response=lambda vars: run_query(vars["query"]),
    )
    | prompt_response
    | llm
    | StrOutputParser()
)

In [23]:
user_question = "Need a recipe using chicken, tomato, basil"
full_sql_chain.invoke({"question": user_question})

"Here's a delightful recipe for **Chicken Caprese Pasta** that combines the flavors of chicken, tomatoes, and fresh basil into a delicious dish. This recipe is perfect for a quick weeknight dinner and is sure to impress!\n\n### Ingredients:\n- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces\n- 2 cups cherry tomatoes, halved\n- 1 cup fresh basil leaves, chopped\n- 3 tablespoons olive oil\n- 3 cloves garlic, minced\n- 8 oz pasta (your choice, such as penne or spaghetti)\n- Salt and pepper to taste\n- Grated Parmesan cheese for serving\n\n### Instructions:\n\n1. **Cook the Pasta**: \n   - Bring a large pot of salted water to a boil. Add the pasta and cook according to package instructions until al dente. Drain and set aside, reserving a cup of pasta water.\n\n2. **Sauté the Chicken**:\n   - In a large skillet, heat 2 tablespoons of olive oil over medium-high heat. Season the chicken pieces with salt and pepper. Add the chicken to the skillet and cook until golden brown

In [24]:
user_question = "What can I cook with rice, potatoes, oranges, carrots, cucumbers?"
full_sql_chain.invoke({"question": user_question})

"With the ingredients you've provided—rice, potatoes, oranges, carrots, and cucumbers—let's create a vibrant and refreshing dish: **Citrus-Infused Rice Bowl with Roasted Vegetables**. This dish will combine the earthiness of the potatoes and carrots with the brightness of oranges and the crunch of cucumbers, all served over a bed of fluffy rice.\n\n### Ingredients:\n- 1 cup rice (any variety you prefer)\n- 2 medium potatoes, diced\n- 2 medium carrots, sliced\n- 1 orange (zest and juice)\n- 1 cucumber, diced\n- Olive oil\n- Salt and pepper\n- Fresh herbs (like parsley or cilantro) for garnish\n\n### Procedure:\n\n1. **Prepare the Rice**: Rinse the rice under cold water until the water runs clear. Cook the rice according to package instructions, using water or broth for added flavor. Once cooked, fluff with a fork and set aside.\n\n2. **Roast the Vegetables**: Preheat your oven to 400°F (200°C). On a baking sheet, toss the diced potatoes and sliced carrots with olive oil, salt, and peppe

In [25]:
user_question = "What can I cook with rice, potatoes, oranges, carrots, cucumbers?"
full_sql_chain.invoke({"question": user_question})

"With the ingredients you've provided—rice, potatoes, oranges, carrots, and cucumbers—I have a delightful and innovative recipe for you: **Citrus Rice Bowl with Roasted Vegetables**. This dish combines the freshness of cucumbers and oranges with the heartiness of rice and roasted potatoes and carrots, creating a vibrant and satisfying meal.\n\n### Ingredients:\n- 1 cup rice (any variety you prefer)\n- 2 medium potatoes, diced\n- 2 medium carrots, sliced\n- 1 orange, juiced and zested\n- 1 cucumber, diced\n- 2 tablespoons olive oil\n- Salt and pepper to taste\n- Fresh herbs (like parsley or cilantro) for garnish\n\n### Procedure:\n\n1. **Prepare the Rice**: \n   - Rinse the rice under cold water until the water runs clear. Cook the rice according to package instructions, using the orange juice instead of water for added flavor. Once cooked, fluff with a fork and set aside.\n\n2. **Roast the Vegetables**: \n   - Preheat your oven to 425°F (220°C). \n   - In a bowl, toss the diced potatoe

In [31]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('LANGSMITH_API_KEY'))
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "PRG RAG App Eval Experiments"

In [32]:
# Ingredients to be used in 5 recipes

user_question_list = [
    "What can I make using bacon, eggs, buns, tomatoes, mayonnaise?",
    "Are there any good dishes to make with bay scallops, butter, onions, parsley, lemons, bread crumbs?",
    "I am looking forward to cook something with rice, chicken, cream of mushroom soup, cream of chicken soup, onion soup, water. Can you please help?",
    "What are my options to cook when I have coriander, allspice, cumin, ground ginger, turmeric, cinnamon, cayenne, pepper, salt, lemon juice, olive oil, garlic, chicken, pita, red onion, parsley, shredded red cabbage, tahini available in my kitcken?",
    "Need to cook something delicious with sour cream, heavy cream, baking powder, kosher salt, sugar, ground black pepper, flour, butter, salt, cutter."]
# ["hamburger", "egg", "bread crumbs", "onion", "milk", "salt", "pepper", "bacon", "catsup"]
# ["stew meat", "cabbage", "onion", "green bell peppers", "oil", "tomatoes", "garlic", "soy sauce"]
# ["chicken broth", "water", "chicken", "onion", "vegetables", "garlic", "salt", "fresh cilantro", "potatoes", "vegetable oil"]
# ["pork sparerib", "black bean sauce", "Chinese rice wine", "cornstarch", "ginger", "garlic", "ground black pepper", "cooking oil", "sesame oil", "sugar", "red chili pepper"]
# ["shell pasta", "red pepper", "green onion", "seafood", "peas", "olive oil", "sesame oil", "rice vinegar", "lemon", "mayonnaise"]

In [37]:
for i in range(len(user_question_list)):
    print(f"{i}: {user_question_list[i]}")
    response = full_sql_chain.invoke({"question": user_question_list[i]})
    print(f"{i}: {response}")
    

0: What can I make using bacon, eggs, buns, tomatoes, mayonnaise
0: With the ingredients you've provided—bacon, eggs, buns, tomatoes, and mayonnaise—let's create a delicious and satisfying dish: **Bacon, Egg, and Tomato Breakfast Sandwiches**.

### Ingredients:
- 4 slices of bacon
- 4 large eggs
- 4 hamburger buns (or any bread of your choice)
- 2 ripe tomatoes, sliced
- Mayonnaise (to taste)
- Fresh lettuce leaves (optional)
- Salt and pepper (to taste)
- Butter (for toasting the buns)

### Procedure:

1. **Cook the Bacon**: In a nonstick frying pan, cook the bacon over medium-high heat until crispy, about 3-4 minutes per side. Once cooked, transfer the bacon to a paper towel-lined plate to drain excess fat.

2. **Fry the Eggs**: In the same pan, reduce the heat to medium-low. Crack the eggs into the pan and season with salt and pepper. Cook the eggs to your desired doneness—sunny-side up, over-easy, or fully cooked.

3. **Toast the Buns**: While the eggs are cooking, spread a little 

In [36]:
user_question = "What can I make using bacon, eggs, buns, tomatoes, mayonnaise"
full_sql_chain.invoke({"question": user_question})

"With the ingredients you've provided—bacon, eggs, buns, tomatoes, and mayonnaise—let's create a delicious and satisfying dish: **Bacon, Egg, and Tomato Breakfast Sandwiches**.\n\n### Ingredients:\n- 4 slices of bacon\n- 4 eggs\n- 4 hamburger buns (or any bread of your choice)\n- 2 ripe tomatoes, sliced\n- Mayonnaise (to taste)\n- Lettuce leaves (optional, for added crunch)\n- Salt and pepper (to taste)\n- Butter (for toasting the buns)\n\n### Procedure:\n\n1. **Cook the Bacon**: In a nonstick frying pan, cook the bacon over medium-high heat until crispy, about 3-4 minutes per side. Once cooked, transfer the bacon to a paper towel-lined plate to drain excess fat.\n\n2. **Fry the Eggs**: In the same pan, reduce the heat to medium-low. Crack the eggs into the pan and season with salt and pepper. Cook them sunny-side up or to your desired doneness, about 2-3 minutes.\n\n3. **Toast the Buns**: While the eggs are cooking, spread a little butter on the cut sides of the hamburger buns. Toast 